In [1]:
import sys
sys.path.append('../Metaheuristic Algorithms/') 
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from DESSA import DESSA  # Importing your DESSA class from the DESSA.py file
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Create an instance of the ImageDataGenerator for data normalization and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Normalize pixel values to be between 0 and 1
    validation_split=0.2  # Use 20% of the data for validation
)

# Define the path to your dataset directory
train_directory = '/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Data/Dataset'

# Setup the training data generator
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(32, 32),  # Resize images to 32x32 for model compatibility
    batch_size=4,  # Smaller batch size for quick testing
    class_mode='categorical',  # Multi-class classification
    subset='training',  # Use the training subset of images
    shuffle=True  # Shuffle the images to reduce model bias
)

# Setup the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(32, 32),
    batch_size=4,  # Consistent batch size with training for simplicity
    class_mode='categorical',
    subset='validation',  # Use the validation subset of images
    shuffle=True
)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [6]:
# Define a function to create a convolutional neural network model based on input parameters
def create_cnn_model(params, input_shape=(32, 32, 3), num_classes=4):
    num_filters, filter_size, dropout_rate = int(params[0]), int(params[1]), float(params[2])
    model = Sequential([
        Conv2D(num_filters, (filter_size, filter_size), padding='same', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Define parameter bounds for the DESSA optimization algorithm
bounds = [(16, 64), (2, 5), (0.1, 0.5)]

# Initialize the DESSA optimizer with a simple objective function
dessa_optimizer = DESSA(lambda params: -create_cnn_model(params).evaluate(validation_generator, steps=1)[1],
                        bounds, population_size=5, iterations=10)

# Run the optimization process
best_params, best_fitness, fitness_history = dessa_optimizer.optimize()

# Print the best parameters and the corresponding fitness value
print("Best Parameters:", best_params)
print("Best Validation Accuracy:", -best_fitness)

1/1 [==============================] - 0s 123ms/step - loss: 1.2588 - accuracy: 0.5000
Best Parameters: [32.26651832  4.41614962  0.38263154]
Best Validation Accuracy: 0.75


In [9]:
# Create the final model using the best parameters found by DESSA
final_model = create_cnn_model(best_params, input_shape=(32, 32, 3), num_classes=4)  # Adjusted num_classes to 4 to match your dataset

# Fit the final model using the training and validation generators
# Here, we are using 'fit_generator' which is recommended for use with data generators
final_history = final_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch.
    epochs=20,  # You can adjust the number of epochs based on how thorough you want the training to be
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Total number of steps (batches of samples) to yield from validation generator per epoch
    verbose=1  # Set verbose to 1 for detailed log output
)

# Print the accuracy of the final model on the test set (using validation data as a proxy for test data)
print("Final model accuracy on validation set:", final_history.history['val_accuracy'][-1])

Epoch 1/20
1280/1280 [==============================] - 12s 9ms/step - loss: 0.9815 - accuracy: 0.5261 - val_loss: 1.1466 - val_accuracy: 0.4146
Epoch 2/20
1280/1280 [==============================] - 11s 9ms/step - loss: 0.8523 - accuracy: 0.5951 - val_loss: 1.2648 - val_accuracy: 0.4161
Epoch 3/20
1280/1280 [==============================] - 11s 9ms/step - loss: 0.7519 - accuracy: 0.6535 - val_loss: 1.3924 - val_accuracy: 0.3660
Epoch 4/20
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6452 - accuracy: 0.7162 - val_loss: 1.3875 - val_accuracy: 0.4365
Epoch 5/20
1280/1280 [==============================] - 14s 11ms/step - loss: 0.5798 - accuracy: 0.7526 - val_loss: 1.5154 - val_accuracy: 0.3926
Epoch 6/20
1280/1280 [==============================] - 11s 9ms/step - loss: 0.4918 - accuracy: 0.7930 - val_loss: 1.3624 - val_accuracy: 0.4828
Epoch 7/20
1280/1280 [==============================] - 11s 9ms/step - loss: 0.4155 - accuracy: 0.8323 - val_loss: 1.3508 - val_a